In [3]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime
from dateutil.relativedelta import relativedelta

court = 320  # Court Precinct 1, Place 2
case_type = "EV"  # Debt Claim
format = "tab"  # Tab-delimited format
start_date = "2020-01-01"
end_date = "2024-09-15"

# Define base URL
base_url = "https://jpwebsite.harriscountytx.gov/PublicExtracts/GetExtractData"

# Helper function to generate date ranges
def generate_date_ranges(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    date_ranges = []

    current = start
    while current <= end:
        start_of_month = current.replace(day=1)
        end_of_month = (start_of_month + relativedelta(months=1)) - relativedelta(days=1)
        date_ranges.append((start_of_month.strftime("%m/%d/%Y"), end_of_month.strftime("%m/%d/%Y")))
        current += relativedelta(months=1)

    return date_ranges

# Generate date ranges for the given period
date_ranges = generate_date_ranges(start_date, end_date)

# Function to download and read data
def download_data(court, fdate, tdate):
    try:
        params = {
            'extractCaseType': 'CV',
            'extract': '7',
            'court': court,
            'casetype': case_type,
            'format': format,
            'fdate': fdate,
            'tdate': tdate
        }
        headers = {
            'Accept': 'text/tab-separated-values'
        }
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        if response.status_code == 200:
            if "attachment" in response.headers.get("Content-Disposition", ""):
                data = pd.read_csv(StringIO(response.text), sep="\t", dtype=str)  # Read tab-delimited file
                if not data.empty and 'Case Number' in data.columns:
                    data = data[data['Case Number'] != 'Case Number']  # Filter out duplicate headers
                    data.drop_duplicates(inplace=True)  # Remove duplicate rows
                    data['court_number'] = court
                    return data
        return None
    except Exception as e:
        return None

# Collect data for the specified court and date ranges
all_data = []
for start, end in date_ranges:
    data = download_data(court, start, end)
    if data is not None:
        all_data.append(data)

# Combine all data
if len(all_data) > 0:
    combined_data = pd.concat(all_data, ignore_index=True)
    # Remove duplicates based on available columns
    #combined_data.drop_duplicates(subset=['Case Number', 'Case Status '], inplace=True)  # Note the space after 'Case Status'
    # Print data to the user
    #print(combined_data)
    # Optionally, save to a CSV file for further analysis
    combined_data.to_csv('JP22_EvictionFiled_Jan20Sep1524_RAW.csv', index=False)
    combined_data.head()
else:
    print("No data was successfully downloaded.")


In [9]:
# Strip excess whitespace from column names
combined_data.columns = combined_data.columns.str.strip()
combined_data['Case File Date'] = pd.to_datetime(combined_data['Case File Date'])

# Filter out cases with a case file date of June 10th, 2024, and before
combined_data = combined_data[combined_data['Case File Date'] > '2024-06-10']
# Get the number of instances for each unique value in 'Case Status'
case_status_counts = combined_data['Case Status'].value_counts()

# Count the number of rows where 'Next Hearing Date' is not empty
next_hearing_date_count = combined_data['Next Hearing Date'].notna().sum()

cases_count = combined_data['Case Number'].notna().sum()
# Display the results
case_status_counts, next_hearing_date_count , cases_count


(Case Status
 Active       352
 Dismissed     10
 Disposed       3
 Appeal         1
 Name: count, dtype: int64,
 19,
 366)

In [11]:
next_hearing_dsc_count = combined_data['Next Hearing Desc'].value_counts()
next_hearing_dsc_count

Next Hearing Desc
Intent to Dismiss for Want of Prosecution    11
Judge Trial                                   4
Judge Compliance                              3
Motion Docket                                 1
Name: count, dtype: int64

In [4]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime
from dateutil.relativedelta import relativedelta

court = 310  # Court Precinct 1, Place 2
case_type = "EV"  # Debt Claim
format = "tab"  # Tab-delimited format
start_date = "2020-01-01"
end_date = "2024-09-15"

# Define base URL
base_url = "https://jpwebsite.harriscountytx.gov/PublicExtracts/GetExtractData"

# Helper function to generate date ranges
def generate_date_ranges(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    date_ranges = []

    current = start
    while current <= end:
        start_of_month = current.replace(day=1)
        end_of_month = (start_of_month + relativedelta(months=1)) - relativedelta(days=1)
        date_ranges.append((start_of_month.strftime("%m/%d/%Y"), end_of_month.strftime("%m/%d/%Y")))
        current += relativedelta(months=1)

    return date_ranges

# Generate date ranges for the given period
date_ranges = generate_date_ranges(start_date, end_date)

# Function to download and read data
def download_data(court, fdate, tdate):
    try:
        params = {
            'extractCaseType': 'CV',
            'extract': '7',
            'court': court,
            'casetype': case_type,
            'format': format,
            'fdate': fdate,
            'tdate': tdate
        }
        headers = {
            'Accept': 'text/tab-separated-values'
        }
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        if response.status_code == 200:
            if "attachment" in response.headers.get("Content-Disposition", ""):
                data = pd.read_csv(StringIO(response.text), sep="\t", dtype=str)  # Read tab-delimited file
                if not data.empty and 'Case Number' in data.columns:
                    data = data[data['Case Number'] != 'Case Number']  # Filter out duplicate headers
                    data.drop_duplicates(inplace=True)  # Remove duplicate rows
                    data['court_number'] = court
                    return data
        return None
    except Exception as e:
        return None

# Collect data for the specified court and date ranges
all_data = []
for start, end in date_ranges:
    data = download_data(court, start, end)
    if data is not None:
        all_data.append(data)

# Combine all data
if len(all_data) > 0:
    combined_data = pd.concat(all_data, ignore_index=True)
    # Remove duplicates based on available columns
    combined_data.drop_duplicates(subset=['Case Number', 'Case Status '], inplace=True)  # Note the space after 'Case Status'
    # Print data to the user
    print(combined_data)
    # Optionally, save to a CSV file for further analysis
    combined_data.to_csv('JP12_EvictionFiled_Jan20Sep1524_RAW.csv', index=False)
    combined_data.head()
else:
    print("No data was successfully downloaded.")


        Case Number Case Type Case Subtype Case File Date  \
0      201200000724        CV     Eviction     01/02/2020   
1      201200001065        CV     Eviction     01/02/2020   
2      201200001066        CV     Eviction     01/02/2020   
3      201200001068        CV     Eviction     01/02/2020   
4      201200001069        CV     Eviction     01/02/2020   
...             ...       ...          ...            ...   
13457  241200351152        CV     Eviction     09/16/2024   
13458  241200351197        CV     Eviction     09/16/2024   
13459  241200351198        CV     Eviction     09/16/2024   
13460  241200351223        CV     Eviction     09/16/2024   
13461  241200351257        CV     Eviction     09/16/2024   

                                          Style Of Case  Nature of Claim   \
0                            Heaney LLC vs. Cedric Gates              NaN   
1      Yale Apartments dba Yale Apartments vs. Marcos...         Eviction   
2      Yale Apartments dba Yale Apar